# Bedrock basic

- 보일러 플레이트용 코드들 소개

---

# 0. Setup


In [5]:
!pip -q install -U boto3 awscli

In [6]:
import json
import boto3

In [7]:
profile_name = None
region = 'us-east-1'

In [8]:
session = boto3.Session(
    profile_name=profile_name,
    region_name=region,
)
bedrock = session.client(service_name='bedrock-runtime')

In [9]:
# modelId = 'anthropic.claude-instant-v1'
modelId = 'anthropic.claude-v2'
accept = 'application/json'
contentType = 'application/json'

# 1. 감성분석기 만들기

- 배송 및 물류에 관한 내용에 대해서만 불만을 찾는 분류기

In [42]:
instruction_prompt = """
You are an agent work for fulfillment center. \
You classify the sentiment conveyed in the user's text only about delivery and inventory matters.
The available sentiment categories are negative, natural, positive.
The user text is enclosed in text tags, <text></text>.

Let's think step by step, and follow below steps to provide a concise response to the user.
Respond the classified sentiment in result tag, <result></result>.
""".strip()

def _build_prompt(chunk):
    user_prompt = f"""
Here is the user text.

<text>{chunk}</text>

Please classify the sentiment, and respond with sentiment only, nothing else.
""".strip()
    return f"""\n\nHuman: {instruction_prompt}\n\nHuman: {user_prompt}\n\n Assistant: """

def generate(chunk):
    body = json.dumps({
        "prompt": _build_prompt(chunk),
        "max_tokens_to_sample": 1024,
        "top_p": 0.9,
        "temperature": 0,
    })
    resp = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    resp_text = resp.get('body').read()
    try:
        return json.loads(resp_text).get('completion')
    except Exception as e:
        print('error occured', e)
        print(resp_text)
        return None

In [43]:
generate("""        
사전예약일 10월 6일 00시 01분 대기를 타고 있다가 00시 01분에 광클하여 00시 10분 정도에 성공해서 아이폰15 일반 256GB 핑크를 구입했습니다
다행히 1차에 성공을 해서 10월 13일 도착 보장이라고 떠있었고,
10월 12일 배송 출발해서 10월 13일 새벽 5시쯤 배송이 완료가 됐습니다.
새벽배송이라고 안써있어서 새벽에 올 줄 몰랐는데,,
일찍와서 너무 좋았어요
아이폰 박스에 뽁뽁이로 포장이 되어 배송이 됐고, 안전하게 배송이 되었습니다.

충전 어댑터가 없이 C to C 케이블만 들어있고, 가격이 비싸다는 단점이...
""")

' <result>positive</result>'

In [44]:
generate("""        
새벽 배송이라고 써있어서 새벽에 올 줄 알았는데, 오후 늦게 도착해서 실망했습니다.
아이폰 박스에 뽁뽁이로 포장이 안되어 있어서 제품 파손이 약간 우려스러웠습니다.
제가 아이폰만 계속 쓰고 있어서 그런것도 있지만, 제품은 정말 최고의 제품입니다.
아직 안바꾸신 분들 계시면 꼭 바꾸시길 추천합니다.
""")

' <result>negative</result>'

In [45]:
generate("""        
I thought it was a dawn delivery since it was mentioned as dawn, but it arrived late in the afternoon, which was disappointing.
Also, I was slightly concerned about product damage because the iPhone box was not securely padded.
Part of it might be due to my continuous use of iPhones, but the product is truly excellent.
If there are still people who haven't switched, I highly recommend doing so.
""")

' <result>negative</result>'